In [1]:
import h5py
import numpy as np
import megengine.data as data
import megengine.data.transform as T
# import megengine
import cv2
import os
from megengine.data.dataset import Dataset
import random

import scipy
from scipy import io

# import megengine as mge
# import megengine.module as M
# import megengine.functional as F

# from megengine.data.transform import ToMode
# from megengine.data import DataLoader, RandomSampler

import torch
import torchvision
import torch.utils.data as Data
# import torch.nn as M
from torch import nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from matplotlib import cm

from tqdm.notebook import tqdm
from torch.autograd import Variable
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hanhan_tie (use `wandb login --relogin` to force relogin)


True

# Configs

In [23]:
class cfg:
    dataset = 'MEG'
    Meg_train_length = 7000    #0-8192
    folder = "/root/autodl-tmp/SIDD_Small_Raw_Only/"
    bs = 64 # batch size
    cut_size = 256
    device = 'cuda:0'
    load_pretrain = False
    lr = 1e-3
    n_steps = 10
    gamma = 0.7
    start_epoch = 0
    n_epochs = 100
    model_name = "model_Autoencoder_MEGDataset_train01"

# Utils

In [ ]:
def psnr(img1, img2):
    mse = F.mse_loss(img1, img2)
#     mse = torch.mean((img1 - img2) ** 2)
    return 20 * torch.log10(1.0 / torch.sqrt(mse)).item()

In [4]:
def toTensor(img):
    img = torch.from_numpy(img)
    return img.float()

# Dataloader

In [9]:
class MEGDataset(Dataset):
    def __init__(self, data_path):
        self.input_path = data_path +  '/competition_train_input.0.2.bin'
        self.gt_path = data_path + '/competition_train_gt.0.2.bin'
        self.content = open(self.input_path, 'rb').read()
        self.input_images = np.frombuffer(content, dtype = 'uint16').reshape((-1,256,256))
        self.content = open(self.gt_path, 'rb').read()
        self.gt_images = np.frombuffer(content, dtype = 'uint16').reshape((-1,256,256))
        print(len(self.input_images))
        
    
    def __getitem__(self, idx):
        inoisy = np.float32([self.input_images[idx, :, :]]) * np.float32(1 / 65536)
        igt = np.float32([self.gt_images[idx, :, :]]) * np.float32(1 / 65536)
        return toTensor(inoisy), toTensor(igt)
    
    def __len__(self):
        return len(self.input_images)

In [10]:
# 
class SIDDDataset(Dataset):
    def __init__(self, mode, data_list):
        self.image_folder = cfg.folder + "new_Data/"
        self.image_list_all = os.listdir(self.image_folder)
        if mode == 1:  #train or val
            data_list_str = ' '.join(data_list)  #全部训练集源文件名组成的字符串
            self.image_list = [img_idx for img_idx in self.image_list_all if img_idx.rpartition('_')[0] in data_list_str]
        else:
            self.image_list = self.image_list_all
        self.cut = cfg.cut_size

    # get the sample
    def __getitem__(self, idx):
        # get the index
        image_id = self.image_list[idx]
        inoisy = scipy.io.loadmat(self.image_folder + self.image_list[idx] + "/GT_RAW_010.MAT")
        igt = scipy.io.loadmat(self.image_folder + self.image_list[idx] + "/NOISY_RAW_010.MAT")
        inoisy = np.float32([np.array(inoisy['var_name'])])
        igt = np.float32([np.array(igt['var_name'])])
        return toTensor(inoisy), toTensor(igt)

    def __len__(self):
        return len(self.image_list)

In [11]:
from torch.utils.data import random_split
a = os.listdir("/root/autodl-tmp/SIDD_Small_Raw_Only/new_Data/")
len(a)

34418

In [ ]:
if cfg.dataset = 'MEG':
    Meg_data_path = '/root/data/burst_raw/'
    dataset = MEGDataset(Meg_data_path)
    train_set, vali_set = random_split(dataset=dataset, lengths=[cfg.Meg_train_length, 8192-cfg.Meg_train_length])
    
elif cfg.dataset = 'SIDD':
    random.seed(32)
    ori_list = os.listdir("/root/autodl-tmp/SIDD_Small_Raw_Only/Data/")
    random.shuffle(ori_list)
    train_list = ori_list[:138]    #0-160，需要根据val_list中的数据数设定计算psnr时的被除数
    val_list = ori_list[138:]
    train_set = SIDDDataset(1, train_list)
    vali_set = SIDDDataset(1, val_list)
    
else:
    print('set an existing dataset')

In [22]:
train_dataloader = Data.DataLoader(train_set,
      shuffle=True, batch_size=cfg.bs
)
vali_dataloader = Data.DataLoader(vali_set,
      shuffle=False, batch_size=1
)

8192


# Net

In [14]:
class Predictor(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(4, 50, 3, padding = 1, bias = True),
            nn.LeakyReLU(negative_slope = 0.125),
            nn.Conv2d(50, 50, 3, padding = 1, bias = True),
            nn.LeakyReLU(negative_slope = 0.125),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(50, 50, 3, padding = 1, bias = True),
            nn.LeakyReLU(negative_slope = 0.125),
            nn.Conv2d(50, 50, 3, padding = 1, bias = True),
            nn.LeakyReLU(negative_slope = 0.125),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(50, 50, 3, padding = 1, bias = True),
            nn.LeakyReLU(negative_slope = 0.125),
            nn.Conv2d(50, 4, 3, padding = 1, bias = True),
            nn.LeakyReLU(negative_slope = 0.125),
        )
    def forward(self, x):
        n, c, h, w = x.shape
        x = x.reshape((n, c, h // 2, 2, w // 2, 2)).permute(0, 1, 3, 5, 2, 4).reshape((n, c * 4, h // 2, w // 2))
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.reshape((n, c, 2, 2, h // 2, w // 2)).permute(0, 1, 4, 2, 5, 3).reshape((n, c, h, w))
        return x

In [15]:
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(4, 16, 3, stride=3, padding=1),  # (b, 16, 10, 10)
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # (b, 16, 5, 5)
            nn.Conv2d(16, 8, 3, stride=2, padding=1),  # (b, 8, 3, 3)
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1)  # (b, 8, 2, 2)
        )
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, 3, stride=2),  # (b, 16, 5, 5)
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 5, stride=2, padding=1),  # (b, 8, 15, 15)
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 4, 4, stride=3, padding=1),  # (b, 1, 28, 28)
            nn.Tanh()
        )
    def forward(self, x):
        n, c, h, w = x.shape
        x = x.reshape((n, c, h // 2, 2, w // 2, 2)).permute(0, 1, 3, 5, 2, 4).reshape((n, c * 4, h // 2, w // 2))
        x = self.encoder(x)
        x = self.decoder(x)
        x = x.reshape((n, c, 2, 2, h // 2, w // 2)).permute(0, 1, 4, 2, 5, 3).reshape((n, c, h, w))
        return x
        


# Train

In [16]:
net = Autoencoder()
total = sum([param.nelement() for param in net.parameters()])
print('  Number of params: %.3fk' % (total / 1e3))

  Number of params: 6.644k


In [17]:
## 调试Autoencoder输出维度
# net = Autoencoder()
# x = torch.randn(1,1,256,256)
# y = net(x)
# print(f'x.shape: {x.shape}')
# print(f'y.shape: {y.shape}')

In [24]:
def train(train_loader, vali_loader):
#     model = Predictor().to(cfg.device)
    model=Autoencoder().to(cfg.device)
    
    # 加载预训练权重
    if cfg.load_pretrain:
        # TODO
        s = 0

    criterion_L1 = torch.nn.L1Loss().to(cfg.device)
    optimizer = torch.optim.Adam([paras for paras in model.parameters() if paras.requires_grad == True], lr=cfg.lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=cfg.n_steps, gamma=cfg.gamma)

    loss_list = []
    
    # 开始训练
    for idx_epoch in range(cfg.start_epoch, cfg.n_epochs):
        # record loss
        loss_epoch = []
        wandb.log({"epoch": idx_epoch})
        for idx_iter, (inoisy, igt) in tqdm(enumerate(train_loader)):
            inoisy, igt = Variable(inoisy).to(cfg.device), Variable(igt).to(cfg.device)
#             print(f'igt.shape: {igt.shape}')
            ipred = model(inoisy)
#             print(len(ipred))
            loss = criterion_L1(ipred, igt)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            loss_epoch.append(loss.data.cpu())
            
        scheduler.step()
        lr_now = scheduler.get_last_lr()
        wandb.log({"lr": float(lr_now[-1])})
        
        # 输出结果
        if idx_epoch % 1 == 0:
            print('Epoch--%4d, loss--%f' %
                  (idx_epoch + 1, float(np.array(loss_epoch).mean())))
            wandb.log({"loss": float(np.array(loss_epoch).mean())})
            
        # 保存模型
        if idx_epoch % 4 == 0:
            torch.save({'epoch': idx_epoch + 1, 'state_dict': model.state_dict()},
                       'log/' + cfg.model_name + '_' + 'epoch' + str(idx_epoch + 1) + '.pth.tar')
            
        # validation
        if idx_epoch % 4 == 0:
            psnr_list = []
            for idx_iter, (inoisy, igt) in tqdm(enumerate(vali_loader)):
                inoisy, igt = Variable(inoisy).to(cfg.device), Variable(igt).to(cfg.device)
                with torch.no_grad():
                    ipred = model(inoisy)
                    
                p = psnr(inoisy, ipred)
                psnr_list.append(p)
            val_len = 8192 - cfg.Meg_train_length
            print("Tested PSNR", str(sum(psnr_list) / val_len))
            wandb.log({"PSNR": float(sum(psnr_list) / val_len)})

In [25]:
train_cfg = {
                "batch size": cfg.bs,
                "cut_size":cfg.cut_size,
                "num_epoch":cfg.n_epochs,
                "init_lr":cfg.lr,
                "n_steps":cfg.n_steps,
                "change_gamma":cfg.gamma,
                "Model Name":cfg.model_name
}

run_name = "A Test Run"

wandb.init(project="blind-denoising",
           config = train_cfg,
           entity="hanhan_tie",
           name = run_name)

wandb.run.save()

train(train_dataloader, vali_dataloader)

PSNR,▁▅▇▆▇▇▇▇████████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr,████▆▆▆▆▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
PSNR,10.13505
epoch,96
loss,0.01086
lr,4e-05


0it [00:00, ?it/s]

Epoch--   1, loss--0.060810


0it [00:00, ?it/s]

Tested PSNR 32.540815487604014


...

Epoch--  98, loss--0.011154


0it [00:00, ?it/s]

Epoch--  99, loss--0.011107


0it [00:00, ?it/s]

Epoch-- 100, loss--0.011094


In [ ]:
# data = scipy.io.loadmat("/root/autodl-tmp/SIDD_Small_Raw_Only/" + "new_Data/" + "0001_001_S6_00100_00060_3200_L_0/NOISY_RAW_010.MAT")
# inoisy = np.float32([np.array(data['var_name'])])
# print(inoisy.shape)

In [ ]:
# inoisy = h5py.File("/root/autodl-tmp/SIDD_Small_Raw_Only/" + "Data/" + "0001_001_S6_00100_00060_3200_L/NOISY_RAW_010.MAT")
# inoisy = np.float32([np.array(inoisy['x'])])
# print(inoisy.shape)

In [ ]:
# class Autoencoder(M.Module):
#     def __init__(self):
#         super().__init__()
#         self.encoder = M.Sequential([
#             # Input(shape=(28, 28, 1,)),
#             M.Conv2D(4, 50, 3, padding = 1, bias = True),
#             M.MaxPooling2D((2, 2), padding='same'),
#             M.Conv2D(32, (3, 3), padding='same', activation='relu',dilation_rate=(3,3)),
#             M.MaxPooling2D((1, 1), padding='same'),
#             M.Conv2D(32, (3, 3), padding='same', activation='relu',dilation_rate=(4,4)),
#             M.MaxPooling2D((1, 1), padding='same'),
#             M.Conv2D(32, (3, 3), padding='same', activation='relu',dilation_rate=(4,4)),
#             M.MaxPooling2D((2, 2), padding='same'),
#         ])
#         self.decoder=M.Sequential([
#             M.Conv2D(32, (3, 3), padding='same', activation='relu',dilation_rate=(2,2)),
#             M.UpSampling2D((2, 2)),
#             M.Conv2D(32, (3, 3), padding='same', activation='relu',dilation_rate=(3,3)),
#             M.UpSampling2D((2, 2)),
#             M.Conv2D(32, (3, 3), padding='same', activation='relu',dilation_rate=(4,4)),
#             M.UpSampling2D((1, 1)),
#             M.Conv2D(32, (3, 3), padding='same', activation='relu',dilation_rate=(4,4)),
#             M.UpSampling2D((1, 1)),
#             M.Conv2D(1, (3, 3), padding='same', activation='sigmoid')
#         ])
#     def forward(self, inputs):
#         e=self.encoder(inputs)
#         y=self.decoder(e)
#         return encoder,decoder

In [ ]:
# # 检查参数量
# model = Predictor()
# # print(model)
# autoencoder=Autoencoder()

In [ ]:
# from megengine.utils.module_stats import module_stats

# input_data = np.random.rand(1, 1, 256, 256).astype("float32")
# total_stats, stats_details = module_stats(
#     net,
#     inputs = (input_data,),
#     cal_params = True,
#     cal_flops = True,
#     logging_to_stdout = True,
# )

# print("params %.3fK MAC/pixel %.0f"%(total_stats.param_dims/1e3, total_stats.flops/input_data.shape[2]/input_data.shape[3]))